In [10]:
class OrderBook:
    def __init__(self, tick_size, lot_size):
     
        # OBJECTIF : Initialisation du carnet d'ordres (tick et lot)
        
        self.tick_size = tick_size
        self.lot_size = lot_size
        self.buy_orders = {}
        self.sell_orders = {}

    
    def add_order(self, order_type, price, quantity, participant_id):
    
        # OBJECTIF : Ajouter un ordre d'achat ou de vente au carnet d'ordres.
 
        # Verifier la validation de l'ordre avec le tick et le lot
        rounded_price = round(price / self.tick_size) * self.tick_size
        
        if quantity % self.lot_size != 0:
            print(f"Erreur: La quantité {quantity} n'est pas un multiple du lot {self.lot_size}.")
            return
        
        # Ajout de l'ordre au dictionnaire correspondant
        if order_type == 'buy':
            if rounded_price in self.buy_orders:
                self.buy_orders[rounded_price].append((quantity, participant_id))
            else:
                self.buy_orders[rounded_price] = [(quantity, participant_id)]
        elif order_type == 'sell':
            if rounded_price in self.sell_orders:
                self.sell_orders[rounded_price].append((quantity, participant_id))
            else:
                self.sell_orders[rounded_price] = [(quantity, participant_id)]

    
    def cancel_order(self, order_type, price, quantity, participant_id):

        # OBJECTIF : Annuler un ordre d'achat ou de vente au carnet d'ordres.
       
        # Verifier la validation de l'ordre avec le tick et le lot
        rounded_price = round(price / self.tick_size) * self.tick_size

        if quantity % self.lot_size != 0:
            print(f"Erreur: La quantité {quantity} n'est pas un multiple du lot {self.lot_size}.")
            return
        
        # Annulation de l'ordre dans le dictionnaire correspondant
        if order_type == 'buy':
            if rounded_price in self.buy_orders:
                order_list = self.buy_orders[rounded_price]
                for order in order_list:
                    if order[1] == participant_id and order[0] == quantity:
                        order_list.remove(order)
                        break
        elif order_type == 'sell':
            if rounded_price in self.sell_orders:
                order_list = self.sell_orders[rounded_price]
                for order in order_list:
                    if order[1] == participant_id and order[0] == quantity:
                        order_list.remove(order)
                        break

    
    def fixing(self):
        
        # OBJECTIF : Determiner le prix d'echange en fixing
        
        # Trier les ordres d'achat et de vente selon leur prix
        sorted_buy_orders = sorted(self.buy_orders.items(), key=lambda x: x[0], reverse=True)
        sorted_sell_orders = sorted(self.sell_orders.items(), key=lambda x: x[0])
    
        # Calculer les quantités cumulées pour chaque prix
        buy_cumulative_quantities = []
        sell_cumulative_quantities = []
    
        buy_quantity = 0
        for price, orders in sorted_buy_orders:
            buy_quantity += sum(order[0] for order in orders)
            buy_cumulative_quantities.append((price, buy_quantity))
    
        sell_quantity = 0
        for price, orders in sorted_sell_orders:
            sell_quantity += sum(order[0] for order in orders)
            sell_cumulative_quantities.append((price, sell_quantity))

        # Determination du prix pour lequel il y a le plus gros volume d'echange
        max_traded_quantity = 0
        fixing_price = None

        buy_index = len(buy_cumulative_quantities) - 1 
        sell_index = 0

        while buy_index >= 0 and sell_index < len(sell_cumulative_quantities):
            buy_price, buy_quantity = buy_cumulative_quantities[buy_index]
            sell_price, sell_quantity = sell_cumulative_quantities[sell_index]
        
            if buy_price == sell_price:
                traded_quantity = min(buy_quantity, sell_quantity)

                if traded_quantity > max_traded_quantity:
                    max_traded_quantity = traded_quantity
                    fixing_price = buy_price
            
                buy_index += 1
                sell_index += 1

            else:
                if buy_price < sell_price:
                    buy_index -= 1
                elif buy_price > sell_price:
                    sell_index += 1
                
    
        return fixing_price



# EXEMPLE

# Initalisation de la class
order_book = OrderBook(tick_size=0.1, lot_size=10)

# Ajout des ordres du carnet d'ordres
order_book.add_order('sell', 52, 1500, 'participant_3')
order_book.add_order('sell', 53, 2500, 'participant_4')
order_book.add_order('sell', 54, 2000, 'participant_5')
order_book.add_order('sell', 55, 2500, 'participant_6')
order_book.add_order('sell', 56, 1600, 'participant_7')
order_book.add_order('sell', 57, 600, 'participant_8')
order_book.add_order('sell', 58, 100, 'participant_14')
order_book.add_order('sell', 60, 10000, 'participant_15')

order_book.add_order('buy', 54, 3000, 'participant_9')
order_book.add_order('buy', 53, 5000, 'participant_10')
order_book.add_order('buy', 52, 3000, 'participant_11')
order_book.add_order('buy', 51, 1000, 'participant_12')
order_book.add_order('buy', 50, 500, 'participant_13')
order_book.add_order('buy', 55, 1500, 'participant_16')
order_book.add_order('buy', 57, 500, 'participant_17')
order_book.add_order('buy', 58, 150, 'participant_18')

# Afficher le fixing d'ouverture
opening_price = order_book.fixing()
print("Le fixing d'ouverture est:",{opening_price})

# Annuler un ordre
order_book.cancel_order('sell',54, 2000, 'participant_5')

#Ajouter un ordre
order_book.add_order('buy', 55, 3000, 'participant_9')

# Affichage du fixing de fermeture
closing_price = order_book.fixing()
print("Le fixing de fermeture est:",{closing_price})

Le fixing d'ouverture est: {54.0}
Le fixing de fermeture est: {55.0}
